In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,roc_auc_score
from sklearn.svm import SVC
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import zscore
import seaborn as sns
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [31]:
file_path = '/content/drive/My Drive/Blossom Academy/ATC Data.csv'
data = pd.read_csv(file_path, encoding='ISO-8859-1')

In [32]:
data.head()

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,MRG,REGULARITY,TOP_PACK,FREQ_TOP_PACK,CHURN
0,dcf68cc2fb515ccad7d8b9b3bd80ee2a4b270063,SAINT-LOUIS,K > 24 month,17000.0,32.0,18000.0,6000.0,34.0,NaN,97.0,355.0,6.0,NaN,NaN,NO,62,All-net 500F=2000F;5d,35.0,0
1,71c44b5ba328db5c4192a80f7cf8f244d9350ed0,NaN,K > 24 month,4300.0,29.0,4427.0,1476.0,37.0,1764.0,8.0,3.0,0.0,NaN,2.0,NO,40,"Data: 100 F=40MB,24H",22.0,0
2,ce46411b1526c94f20a383b8cb188f8d27f82a0a,TAMBACOUNDA,K > 24 month,1500.0,3.0,1500.0,500.0,3.0,NaN,30.0,30.0,NaN,NaN,NaN,NO,32,All-net 500F=2000F;5d,3.0,0
3,f467cdb6669818373c26c2bad44e01ba66f97d21,FATICK,K > 24 month,1500.0,3.0,2497.0,832.0,4.0,0.0,159.0,45.0,19.0,NaN,NaN,NO,18,On net 200F=Unlimited _call24H,3.0,0
4,ec45e1a1888a32b5dcce0954cfec20c6e037db31,FATICK,K > 24 month,NaN,NaN,498.0,166.0,3.0,1.0,1.0,3.0,NaN,NaN,NaN,NO,50,NaN,NaN,0


In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 19 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user_id         400000 non-null  object 
 1   REGION          242480 non-null  object 
 2   TENURE          400000 non-null  object 
 3   MONTANT         259723 non-null  float64
 4   FREQUENCE_RECH  259723 non-null  float64
 5   REVENUE         265337 non-null  float64
 6   ARPU_SEGMENT    265337 non-null  float64
 7   FREQUENCE       265337 non-null  float64
 8   DATA_VOLUME     203146 non-null  float64
 9   ON_NET          254181 non-null  float64
 10  ORANGE          233683 non-null  float64
 11  TIGO            160614 non-null  float64
 12  ZONE1           31690 non-null   float64
 13  ZONE2           25513 non-null   float64
 14  MRG             400000 non-null  object 
 15  REGULARITY      400000 non-null  int64  
 16  TOP_PACK        232671 non-null  object 
 17  FREQ_TOP_P

In [34]:
pd.unique(data['REGION'])

array(['SAINT-LOUIS', nan, 'TAMBACOUNDA', 'FATICK', 'THIES', 'LOUGA',
       'DAKAR', 'KAFFRINE', 'DIOURBEL', 'KOLDA', 'MATAM', 'SEDHIOU',
       'KAOLACK', 'ZIGUINCHOR', 'KEDOUGOU'], dtype=object)

In [35]:
pd.unique(data['TENURE'])

array(['K > 24 month', 'H 15-18 month', 'G 12-15 month', 'J 21-24 month',
       'I 18-21 month', 'E 6-9 month', 'F 9-12 month', 'D 3-6 month'],
      dtype=object)

In [36]:
pd.unique(data['MRG'])

array(['NO'], dtype=object)

In [37]:
pd.unique(data['TOP_PACK'])

array(['All-net 500F=2000F;5d', 'Data: 100 F=40MB,24H',
       'On net 200F=Unlimited _call24H', nan, 'Data:490F=1GB,7d',
       'Data:1000F=5GB,7d', 'VAS(IVR_Radio_Daily)',
       'Data:200F=Unlimited,24H', 'Jokko_Daily',
       'Mixt 250F=Unlimited_call24H', 'Data: 200 F=100MB,24H',
       'MIXT:500F= 2500F on net _2500F off net;2d', 'Data:1000F=2GB,30d',
       'IVR Echat_Daily_50F', 'On-net 1000F=10MilF;10d',
       'All-net 600F= 3000F ;5d', 'Twter_U2opia_Daily',
       'MIXT: 200mnoff net _unl on net _5Go;30d', 'On-net 500F_FNF;3d',
       'Twter_U2opia_Weekly', 'All-net 500F =2000F_AllNet_Unlimited',
       'Yewouleen_PKG', 'On-net 500=4000,10d', 'On-net 200F=60mn;1d',
       'Data:3000F=10GB,30d', 'Incoming_Bonus_woma',
       'All-net 1000=5000;5d', 'Data:500F=2GB,24H', 'Data:300F=100MB,2d',
       'MIXT: 390F=04HOn-net_400SMS_400 Mo;4h\t', 'Data:50F=30MB_24H',
       'All-net 1000F=(3000F On+3000F Off);5d',
       'All-net 500F=1250F_AllNet_1250_Onnet;48h',
       'Data:150F=

In [38]:
# is null?
print(data.isnull().sum())

user_id                0
REGION            157520
TENURE                 0
MONTANT           140277
FREQUENCE_RECH    140277
REVENUE           134663
ARPU_SEGMENT      134663
FREQUENCE         134663
DATA_VOLUME       196854
ON_NET            145819
ORANGE            166317
TIGO              239386
ZONE1             368310
ZONE2             374487
MRG                    0
REGULARITY             0
TOP_PACK          167329
FREQ_TOP_PACK     167329
CHURN                  0
dtype: int64


In [39]:
data.drop(columns=['TIGO', 'ZONE1', "ZONE2", 'MRG'], inplace=True)

In [40]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user_id         400000 non-null  object 
 1   REGION          242480 non-null  object 
 2   TENURE          400000 non-null  object 
 3   MONTANT         259723 non-null  float64
 4   FREQUENCE_RECH  259723 non-null  float64
 5   REVENUE         265337 non-null  float64
 6   ARPU_SEGMENT    265337 non-null  float64
 7   FREQUENCE       265337 non-null  float64
 8   DATA_VOLUME     203146 non-null  float64
 9   ON_NET          254181 non-null  float64
 10  ORANGE          233683 non-null  float64
 11  REGULARITY      400000 non-null  int64  
 12  TOP_PACK        232671 non-null  object 
 13  FREQ_TOP_PACK   232671 non-null  float64
 14  CHURN           400000 non-null  int64  
dtypes: float64(9), int64(2), object(4)
memory usage: 45.8+ MB


In [41]:
data['REGULARITY'] = data['REGULARITY'].astype('float64')
print(data.dtypes)

user_id            object
REGION             object
TENURE             object
MONTANT           float64
FREQUENCE_RECH    float64
REVENUE           float64
ARPU_SEGMENT      float64
FREQUENCE         float64
DATA_VOLUME       float64
ON_NET            float64
ORANGE            float64
REGULARITY        float64
TOP_PACK           object
FREQ_TOP_PACK     float64
CHURN               int64
dtype: object


In [42]:
numerical_columns = data.select_dtypes(include=['float64']).columns
numerical_impute = SimpleImputer(strategy='median')
for col in numerical_columns:
  data[col] = numerical_impute.fit_transform(data[[col]])

In [43]:
categorical_columns = data.select_dtypes(include=['object']).columns
for col in categorical_columns:
  data[col] = data[col].fillna(data[col].mode()[0])

In [44]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user_id         400000 non-null  object 
 1   REGION          400000 non-null  object 
 2   TENURE          400000 non-null  object 
 3   MONTANT         400000 non-null  float64
 4   FREQUENCE_RECH  400000 non-null  float64
 5   REVENUE         400000 non-null  float64
 6   ARPU_SEGMENT    400000 non-null  float64
 7   FREQUENCE       400000 non-null  float64
 8   DATA_VOLUME     400000 non-null  float64
 9   ON_NET          400000 non-null  float64
 10  ORANGE          400000 non-null  float64
 11  REGULARITY      400000 non-null  float64
 12  TOP_PACK        400000 non-null  object 
 13  FREQ_TOP_PACK   400000 non-null  float64
 14  CHURN           400000 non-null  int64  
dtypes: float64(10), int64(1), object(4)
memory usage: 45.8+ MB


In [45]:
# detect columns with outliers
numerical_columns = data.select_dtypes(include=['float64']).columns
outliers = {}
for col in numerical_columns:
    z_scores = zscore(data[col])
    outliers[col] = data[abs(z_scores) > 3]
print(outliers.keys())

dict_keys(['MONTANT', 'FREQUENCE_RECH', 'REVENUE', 'ARPU_SEGMENT', 'FREQUENCE', 'DATA_VOLUME', 'ON_NET', 'ORANGE', 'REGULARITY', 'FREQ_TOP_PACK'])


In [46]:
scaler = RobustScaler()
for col in numerical_columns:
  data[col] = scaler.fit_transform(data[[col]])

In [47]:
label = LabelEncoder()
for col in categorical_columns:
  data[col] = label.fit_transform(data[col])

In [48]:
print(data.columns)

Index(['user_id', 'REGION', 'TENURE', 'MONTANT', 'FREQUENCE_RECH', 'REVENUE',
       'ARPU_SEGMENT', 'FREQUENCE', 'DATA_VOLUME', 'ON_NET', 'ORANGE',
       'REGULARITY', 'TOP_PACK', 'FREQ_TOP_PACK', 'CHURN'],
      dtype='object')


In [49]:
# Define features and target variable
X = data.drop(columns=['user_id', 'CHURN'])
y = data[['CHURN']]


In [50]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [51]:
# Build and train the Decision Tree model
model = RandomForestClassifier(random_state = 0)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

In [52]:
# Evaluate the model
y_pred = model.predict(X_test)      ##We won't predict with accuracy as the data is not balanced, rather we use ROC or F1score.
y_pred_prob = model.predict_proba(X_test)[:, 1]  ## you predict in terms of probablity for classification problems.

In [53]:
y_pred

array([0, 0, 0, ..., 1, 0, 0])

In [54]:
# y_test, y_pred, and y_pred_prob are defined
conf_matrix = confusion_matrix(y_test, y_pred)  ##Confusion matrix
roc_auc = roc_auc_score(y_test, y_pred_prob)    ##ROC accuracy
accuracy = accuracy_score(y_test,y_pred)        ## Normal accuracy score

print("Confusion Matrix:\n", conf_matrix)
print("ROC AUC Score:", round(roc_auc,2))
print("Accuracy : " + str(round(accuracy,2) * 100) + " %")

Confusion Matrix:
 [[60134  4889]
 [ 5766  9211]]
ROC AUC Score: 0.9
Accuracy : 87.0 %


In [55]:
# Print the classification report
classification_report_output = classification_report(y_test, y_pred)
print(classification_report_output)

              precision    recall  f1-score   support

           0       0.91      0.92      0.92     65023
           1       0.65      0.62      0.63     14977

    accuracy                           0.87     80000
   macro avg       0.78      0.77      0.78     80000
weighted avg       0.86      0.87      0.87     80000



In [56]:
# Save the model to a file
joblib.dump(model, "Capstone_Proj.pkl")


['Capstone_Proj.pkl']